In [1]:
import numpy as np
import matplotlib.pyplot as plt
from TSKY import *
from g_by_Tsys import *

In [2]:
data=np.loadtxt('samples.txt',dtype='object')

In [3]:
PSR = data[:,1]
S_400 = data[:,2]
S_1400 = data[:,3]
period = data[:,4].astype('float')
width = data[:,5]
Gl = data[:,6]
Gb = data[:,7]
BW = data[:,8].astype('float')
N_channel_B3 = data[:,9].astype('float')
N_channel_B5 = data[:,10].astype('float')
DM = data[:,11].astype('float')
T_obs=data[:,12].astype('float')
data_b3 = np.loadtxt('band3.par')
data_b5 = np.loadtxt('band5.par')
n_chan=1024

In [4]:
tskylist = readtskyfile()

In [5]:
def ISM_WIDTH(N_channel, bandwidth, nu, DM_psr, width_in):
    Chan_width = bandwidth/N_channel # in MHz
    K_DM = 8.3*(10.0**6.0)
    W_DM = K_DM*DM_psr*(Chan_width/(nu**3.0))
    W_scatt = 10.0**(-6.46+0.154*np.log10(DM_psr) + 1.07*(np.log10(DM_psr))**2.0 -3.86*np.log10(nu/1000.0))
    if width_in == '*':
        effective_width = '*'
    else:
        effective_width = np.sqrt(W_DM**2.0 + float(width_in)**2.0 + W_scatt**2.0)

    return effective_width

In [6]:
def SNR(bandwidth, Int_time,flux,period, width_in, nu, BAND, N_channel, T_sky, DM_psr):

    width_eff = ISM_WIDTH(N_channel,bandwidth,nu,DM_psr,width_in)

    if flux != '*' and width_eff !='*':
        flux = flux*10**-3
        width = float(width_eff)*10**-3 # conversting from ms to sec
        N_pol = 2.0
        Delta_nu = bandwidth*10**6.0 ## Converting MHz to Hz
        Int_time = Int_time*60.0 ## converting it in seconds
        if BAND =='BAND3':
            N_PA = 10.0
            #G_by_Tsys =0.32/(106.0+T_sky)
            G_by_Tsys = g_by_t_sys(data_b3, nu)/(1.0 + T_sky/106.0)
        if BAND == 'BAND5':
            N_PA = 14.0
            #G_by_Tsys = 0.22/(73.0+T_sky)
            G_by_Tsys = g_by_t_sys(data_b5, nu)/(1.0 + T_sky/73.0)
        SNR = (flux*N_PA)*G_by_Tsys*(np.sqrt(N_pol*Delta_nu*Int_time))*(np.sqrt((period-width)/width))

    else:
        SNR = 0.0

    return SNR

In [7]:
logical = np.logical_or(S_1400=='*', S_400=='*')
logical_flux = np.logical_and(S_1400=='*', S_400=='*')

In [8]:
Band3_T=[]
Band5_T=[]
T_sky_400_l= []
T_sky_1360_l =[]

In [9]:
for i in range(len(PSR)):
    #print (i)
    if logical[i] == True:
        #print ('LEV 1')
        spectral_index = -1.6
    if logical[i] == False:
        #print ('LEV 2')
        S_400_m = float(S_400[i])
        S_1400_m = float(S_1400[i])
        spectral_index =np.log(S_400_m/S_1400_m)/np.log(400.0/1400.0)
    
    if logical_flux[i] == True:
        S_BAND5 ='*'
        S_BAND3 ='*'
    else:
        if S_1400[i] != '*':
            S_BAND5 = float(S_1400[i])*(1360.0/1400.0)**(spectral_index)
            S_BAND3 = float(S_1400[i])*(400.0/1400.0)**(spectral_index)
        if S_1400[i] == '*':
            S_BAND5 = float(S_400[i])*(1360.0/400.0)**(spectral_index)
            S_BAND3 = float(S_400[i])*(400.0/400.0)**(spectral_index)


    if Gl[i] =='*' or Gb[i]=='*':
            T_sky = 0.0
    else:
            T_sky_400 = tskypy(400.0, float(Gl[i]), float(Gb[i]), tskylist) ## Computing T_sky from Manjari Bagchi's code.

            T_sky_1360 = tskypy(1360.0, float(Gl[i]), float(Gb[i]), tskylist) ## Computing T_sky from Manjari Bagchi's code.
    

    T_sky_400_l.append(T_sky_400)
    T_sky_1360_l.append(T_sky_1360)
    Band3_T.append(SNR(BW[i],T_obs[i],S_BAND3,period[i], width[i], 400.0, 'BAND3', n_chan, T_sky_400, DM[i]))
    Band5_T.append(SNR(BW[i],T_obs[i],S_BAND5,period[i], width[i], 1360.0, 'BAND5', n_chan, T_sky_1360, DM[i]))
    

/tmp/ipykernel_1102235/454316610.py:19: RuntimeWarning: invalid value encountered in sqrt
  SNR = (flux*N_PA)*G_by_Tsys*(np.sqrt(N_pol*Delta_nu*Int_time))*(np.sqrt((period-width)/width))


In [10]:

array = np.array(list(zip(PSR,T_obs,Band3_T, Band5_T)), dtype=object)

In [11]:
np.savetxt('snr_output.txt', array, delimiter='\t', fmt=['%s', '%0.f','%4.2f', '%4.2f'],
          header='#J-Name\tobs_time(min)\tS/N(B3)\tS/N(B5))')

In [12]:
np.loadtxt('snr_output.txt',dtype=object)

array([['J0030+0451', '32', '581.95', '61.36'],
       ['J0034-0534', '32', '842.83', '32.53'],
       ['J0437-4715', '10', '52652.73', '19482.41'],
       ['J0613-0200', '20', '258.52', '154.48'],
       ['J0740+6620', '20', '1594.56', '108.41'],
       ['J0751+1807', '20', '328.80', '183.89'],
       ['J0900-3144', '20', '945.91', '315.49'],
       ['J1012+5307', '15', '1301.51', '221.58'],
       ['J1022+1001', '20', '6750.88', '588.17'],
       ['J1125+7819', '20', '996.91', '97.74'],
       ['J1600-3053', '30', '514.21', '512.74'],
       ['J1643-1224', '25', '2069.05', '570.33'],
       ['J1713+0747', '32', '424.15', '1987.53'],
       ['J1730-2304', '27', '1702.66', '357.81'],
       ['J1744-1134', '27', '1456.47', '561.05'],
       ['J1857+0943', '32', '824.60', '425.53'],
       ['J1909-3744', '32', '1340.31', '743.85'],
       ['J1939+2134', '10', 'nan', '1550.67'],
       ['J1944+0907', '30', '971.60', '238.26'],
       ['J2124-3358', '30', '1266.95', '462.87'],
       ['J21